# Coverage simulation in Arc de Triomphe in Paris


<img src="Screenshot 2025-03-24 at 7.52.38 PM.png" alt="drawing" width="500"/>

Credit Sionna RT: The scene was created with data downloaded from [OpenStreetMap](https://www.openstreetmap.org/) and the help of [Blender](https://www.blender.org/) and the [Blender-OSM](https://github.com/vvoovv/blender-osm) and [Mitsuba Blender](https://github.com/mitsuba-renderer/mitsuba-blender) add-ons. The data is licensed under [the Open Data Commons Open Database License (ODbL)](https://openstreetmap.org/copyright).

this is a tutotial adopted from: https://colab.research.google.com/github/nvlabs/sionna-rt/blob/main/tutorials/Radio-Maps.ipynb#scrollTo=7BEQgaFJbcJ1 

## usecase

**Role name: ** RF optimization engineers. 

**Proejct:** 
In telco providers, there are multiple RF planning and optimization teams that plan, implement and test changes in RF network. 

Change variables can be: 
- antenna location
- antenna height
- antenna array system
- antenna orientation(tilt and azimuth) 
- Tx power

In this simulation we simulate the effect of upgrading antenna array systems of the network from config 1 to config 2 in 28GHz. 
- Config 1: transmitter: [8x2 UPA], user: [2x2 UPA]
- Config 2: transmitter: [16x16 UPA], user: [2x2 UPA]

UPA: uniform planar array 

In [0]:
%python
# Install the required packages
%pip install drjit mitsuba sionna-rt

In [0]:
import numpy as np
import drjit as dr
import mitsuba as mi
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

from sionna.rt import LambertianPattern, DirectivePattern, BackscatteringPattern,\
                      load_scene, Camera, Transmitter, Receiver, PlanarArray,\
                      PathSolver, RadioMapSolver, cpx_abs, cpx_convert

no_preview = True # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization


from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera, watt_to_dbm,\
                      RadioMapSolver, PathSolver

In both scnarios these variables are fixed

- we have 7 cells 
- 50 users per cell
- fixed antenna location (specified below)
- fixed antenna height (specified below)
- fixed antenna power (specified below)
- fixed antenna orientation (specified below)
- fixed bandwidth 100 MHz (specified below)
- fixed carrier frequency 28 GHz (specified below)



 

In [0]:
class SceneConfiguration:
  def __init__(self, num_rows_tx, num_cols_tx, num_rows_rx, num_cols_rx):
        self.num_rows_tx = num_rows_tx
        self.num_cols_tx = num_cols_tx
        self.num_rows_rx = num_rows_rx
        self.num_cols_rx = num_cols_rx
        # Place transmitters
        self.positions = np.array( #lat, long, height
                  [[-150.3, 21.63, 42.5],
                    [-125.1, 9.58, 42.5],
                    [-104.5, 54.94, 42.5],
                    [-128.6, 66.73, 42.5],
                    [172.1, 103.7, 24],
                    [232.8, -95.5, 17],
                    [80.1, 193.8, 21]
                  ])
        self.look_ats = np.array( #orientation
                  [[-216, -21,0],
                  [-90, -80, 0],
                  [-16.5, 75.8, 0],
                  [-164, 153.7, 0],
                  [247, 92, 0],
                  [211, -180, 0],
                  [126.3, 194.7, 0]
                  ])
        self.power_dbms = [23, 23, 23, 23, 23, 23, 23] #power

  def config_scene(self):
      """Load and configure a scene"""
      scene = load_scene(sionna.rt.scene.etoile)
      scene.bandwidth=100e6
      scene.frequency = 2.8e9

      # Configure antenna arrays for all transmitters and receivers
      scene.tx_array = PlanarArray(num_rows=self.num_rows_tx,
                                  num_cols=self.num_cols_tx,
                                  pattern="tr38901",
                                  polarization="V")

      scene.rx_array = PlanarArray(num_rows=self.num_rows_rx,
                                  num_cols=self.num_cols_rx,
                                  pattern="tr38901",
                                  polarization="V")



      for i, position in enumerate(self.positions):
          scene.add(Transmitter(name=f'tx{i}',
                                position=position,
                                look_at=self.look_ats[i],
                                power_dbm=self.power_dbms[i]))

      return scene

## Config 1

We have a cell transmitter with [8x2 UPA] and user cellphone with [2x2 UPA]. It means antenna array system of transmitter has 16 components arranged in 8 by 2 rectangular shape, and antenna array system of Cell phone has 4 components arranged in 2 by 2 square shape. 

<img src="./Screenshot 2025-03-24 at 7.37.02 PM.png" alt="Config 1" width="300"/>

In [0]:

rm_solver = RadioMapSolver()

# Load and configure scene
scene_config = SceneConfiguration(num_rows_tx=16, num_cols_tx=16, num_rows_rx=2, num_cols_rx=2)
scene = scene_config.config_scene()
# Compute the SINR map
rm_etoile = rm_solver(scene,
                      max_depth=5,
                      samples_per_tx=10**7,
                      cell_size=(1, 1))